
### RUHANI FAIHEEM RAHMAN
### 801126769

For this assignment, testing result is really time consuming. I can not get any results using the 1000 sentences given in our corpus, That's why I use sorther version of the corpus. I set a variable 'n' to select the sentence size. In this notebook, I use n=10 sentence for quick iteration. We can change the value of 'n' in the next cell to run the notebook with higher corpus.

In [1]:
n = 10

In [2]:
from operator import itemgetter
from copy import deepcopy

In [3]:
'''
This method will read the train data and reutrn the translation pair of the two language in a list of dictionary format.
{'fr':'Reprise de la session','en':'Resumption of the session'}
'''
def get_corpus(file_1, file_2, label_1, label_2, n):
    with open(file_1, 'r') as f:
        corpus1 = f.read()
        # the following line will remove the unnecessary characters from the corpus
        corpus1 = corpus1.translate(str.maketrans('', '', '!"#$%&\()*+,./:;<=>?@[\\]^_`{|}~0123456789'))
        corpus1 = corpus1.split('\n')[:n]
    with open(file_2, 'r') as f:
        corpus2 = f.read()
        corpus2 = corpus2.translate(str.maketrans('', '', '!"#$%&\()*+,./:;<=>?@[\\]^_`{|}~0123456789'))
        corpus2 = corpus2.split('\n')[:n]
    corpus_list = list()
    for corpus in zip(corpus1,corpus2):
        temp_dict = {label_1:corpus[0],label_2:corpus[1]}
        corpus_list.append(temp_dict)
#     print(len(corpus1))
#     print(len(corpus2))
    return corpus_list

In [4]:
'''
This method will return all the words for a language in a dictinary format.
'''
def get_words(corpus):
    def source_words(lang):
        for pair in corpus:
            for word in pair[lang].split():
                yield word
    return {lang: set(source_words(lang)) for lang in ('en', 'fr')}


In [5]:
'''
Initialize the translation probability table. First get all the words then calculate initial probability for all of them.
'''
def init(corpus, words):
#     words = get_words(corpus)
    return_dict = {word_en: {word_fr: 1/len(words['en']) for word_fr in words['fr']} for word_en in words['en']}
    return return_dict

In [6]:
'''
This is the main algorithm for calculating ransition probabilities. We implemented IBM 1 descibied in the PDF.
'''
def step(corpus, words, total_s, prev_table):
    cur_table = deepcopy(prev_table)

    counts = {word_en: {word_fr: 0 for word_fr in words['fr']} for word_en in words['en']}

    totals = {word_fr: 0 for word_fr in words['fr']}

    for (es, fs) in [(pair['en'].split(), pair['fr'].split()) for pair in corpus]:
        for e in es:
            total_s[e] = 0
            for f in fs:
                total_s[e] += cur_table[e][f]

        for e in es:
            for f in fs:
                counts[e][f] += (cur_table[e][f] / total_s[e])
                totals[f] += cur_table[e][f] / total_s[e]

    for f in words['fr']:
        for e in words['en']:
            cur_table[e][f] = counts[e][f] / totals[f]

    return cur_table

In [ ]:
'''
Checks if the table is converged or not.
'''
def is_converged(prev_table, curr_table, epsilon):
    delta = distance(prev_table, curr_table)
    return delta < epsilon


In [ ]:
'''
uses root mean square error to find the distance
'''
def distance(prev, curr):
    diff = 0
    for key, val in prev.items():
        for keys, vals in val.items():
            diff += (prev[key][keys] - curr[key][keys]) ** 2
#     print(diff ** 0.5)
    return diff ** 0.5


In [ ]:
'''
IBM Model 1
'''
def ibm_model(corpus, epsilon):

    words = get_words(corpus)

    total_s = {word_en: 0 for word_en in words['en']}
#     print(total_s)
    prev_table = init(corpus, words)

    converged = False
    iterations = 0
    while not converged:
        curr_table = step(corpus, words, total_s, prev_table)

        converged = is_converged(prev_table, curr_table, epsilon)
        prev_table = curr_table
        iterations += 1

    print('Total Iterations: ', iterations)
    return curr_table

In [ ]:
'''
Print Summary results of the translation probabilities. Prints the sorted list.
'''
def summarize(table):
    for key, vals in table.items():
        print(key, end='\n')
        print(list(sorted(vals.items(), key=itemgetter(1), reverse=True)))
        print('\n\n')

In [ ]:
'''
Gets the word which has highest probabilities.
'''
def maxm(m_key, dict, sent):
    h_max = 0
    alligned = ''
    # print(dict)
    # print(sent)
    for key, val in dict.items():
        if val >= h_max and key in sent:
            # print(key, val)
            h_max = val
            alligned = key
    print(m_key, ':', alligned, ":", h_max)
    return alligned

In [ ]:
'''
Find allignment in a sentence of two separate language
'''
def find_allignment(corpus, result_table):
    for sent in corpus:
        print('\n\n', sent['en'],'\n',sent['fr'],'\n\n')
        for en in sent['en'].split():
            # print(en)
            maxm(en, result_table[en], sent['fr'].split())


#### Problem 01

For this task, I pick French vs English corpus. This problem requirs to print the transition probabilities. We use IBM Model to calculate the traslation probabilities. We use Dictionary Dataset here.

In [ ]:
'''
Getting all the problem 01 together by calling this method.
'''
def task1(file_1='data/de-en.de', file_2='data/de-en.en', label_1='fr', label_2='en', epsilon=0.001, n=10):
# def main(file_1='data/fr-en.fr', file_2='data/fr-en.en', label_1='fr', label_2='en', *, outfile:'o'=None, epsilon:'e'=0.001, verbose:'v'=False):

    corpus = get_corpus(file_1, file_2, label_1, label_2, n)

    probability_table = ibm_model(corpus, epsilon)

    summarize(probability_table)


Results of transition probability table in French vs English Language

In [ ]:
task1(file_1='data/fr-en.fr', file_2='data/fr-en.en', label_1='fr', label_2='en', epsilon=0.001, n=n)

Total Iterations:  292
Resumption
[('Reprise', 0.6415158723929386), ('session', 2.2338685692166745e-78), ('la', 4.408703917132715e-117), ('de', 1.4261756596852796e-219), ("l'", 0.0), ('dÃ©bat', 0.0), ('jours', 0.0), ('toutes', 0.0), ('a', 0.0), ('nous', 0.0), ('vux', 0.0), ('ne', 0.0), ('ont', 0.0), ('europÃ©enne', 0.0), ('reprise', 0.0), ('probablement', 0.0), ("c'", 0.0), ('vacances', 0.0), ('les', 0.0), ('pÃ©riode', 0.0), ('Madame', 0.0), ('Je', 0.0), ('diffÃ©rents', 0.0), ('catastrophes', 0.0), ('plusieurs', 0.0), ('renouvelle', 0.0), ('PrÃ©sidente', 0.0), ('nombre', 0.0), ('europÃ©en', 0.0), ('Kumar', 0.0), ('certain', 0.0), ('procÃ©dure', 0.0), ('me', 0.0), ('en', 0.0), ('par', 0.0), ('des', 0.0), ('personnes', 0.0), ('touchÃ©s', 0.0), ('avait', 0.0), ('Sri', 0.0), ('motion', 0.0), ('Comme', 0.0), ('une', 0.0), ('tÃ©lÃ©vision', 0.0), ('tempÃªtes', 0.0), ('tous', 0.0), ('Lanka', 0.0), ('pays', 0.0), ('le', 0.0), ('espÃ©rant', 0.0), ('Vous', 0.0), ('M', 0.0), ('dÃ©clare', 0.0), ('p

Results of transition probability table in German vs English Language

In [ ]:
task1(file_1='data/de-en.de', file_2='data/de-en.en', label_1='fr', label_2='en', epsilon=0.001, n=n)

Total Iterations:  220
Resumption
[('Wiederaufnahme', 0.510694573454118), ('Sitzungsperiode', 1.352731165585297e-52), ('der', 6.905286116415044e-145), ('wissen', 0.0), ('und', 0.0), ('auch', 0.0), ('ist', 0.0), ('Millenium-Bug', 0.0), ('die', 0.0), ('schÃ¶ne', 0.0), ('hatte', 0.0), ('in', 0.0), ('erheben', 0.0), ('vor', 0.0), ('besteht', 0.0), ('Gute', 0.0), ('konnten', 0.0), ('ich', 0.0), ('jÃ¼ngster', 0.0), ('Toten', 0.0), ('einiger', 0.0), ('LÃ¤ndern', 0.0), ('Freitag', 0.0), ('Kumar', 0.0), ('Besuch', 0.0), ('geworden', 0.0), ('des', 0.0), ('Im', 0.0), ('alles', 0.0), ('gefÃ¼rchtete', 0.0), ('Sri', 0.0), ('Sie', 0.0), ('Parlaments', 0.0), ('Opfern', 0.0), ('GeschÃ¤ftsordnung', 0.0), ('Frau', 0.0), ('im', 0.0), ('Bombenexplosionen', 0.0), ('Tagen', 0.0), ('feststellen', 0.0), ('unserer', 0.0), ('Lanka', 0.0), ('wiederaufgenommen', 0.0), ('hoffe', 0.0), ('Zu', 0.0), ('zur', 0.0), ('sind', 0.0), ('Wunsch', 0.0), ('aus', 0.0), ('Fernsehen', 0.0), ('Schweigeminute', 0.0), ('Wie', 0.0), 

#### Problem 02

Finds the sentence allignment by calling this method.

In [ ]:
def task2(file_1='data/de-en.de', file_2='data/de-en.en', label_1='fr', label_2='en', epsilon=0.001, n=10):
# def main(file_1='data/fr-en.fr', file_2='data/fr-en.en', label_1='fr', label_2='en', *, outfile:'o'=None, epsilon:'e'=0.001, verbose:'v'=False):

    corpus = get_corpus(file_1, file_2, label_1, label_2, n)

    probability_table = ibm_model(corpus, epsilon)

    find_allignment(corpus,probability_table)


Allignment results for French vs Englsih

In [ ]:
task2(file_1='data/fr-en.fr', file_2='data/fr-en.en', label_1='fr', label_2='en', epsilon=0.001, n=n)

Total Iterations:  292


 Resumption of the session 
 Reprise de la session 


Resumption : Reprise : 0.6415158723929386
of : de : 0.5982498384525219
the : session : 0.7169882728862997
session : Reprise : 0.3584841276070614


 I declare resumed the session of the European Parliament adjourned on Friday  December   and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period  
 Je dÃ©clare reprise la session du Parlement europÃ©en qui avait Ã©tÃ© interrompue le vendredi  dÃ©cembre dernier et je vous renouvelle tous mes vux en espÃ©rant que vous avez passÃ© de bonnes vacances  


I : je : 0.4656191010094766
declare : passÃ© : 0.047463472119781996
resumed : passÃ© : 0.047463472119781996
the : qui : 0.8856689640217726
session : session : 0.28301172711370004
of : de : 0.5982498384525219
the : qui : 0.8856689640217726
European : avait : 0.438980467824816
Parliament : avait : 0.5260519749195937
adjourned : passÃ© : 0.047463472119781996
on : j

#### Problem 03

Allignment results for German vs Englsih

In [ ]:
task2(file_1='data/de-en.de', file_2='data/de-en.en', label_1='fr', label_2='en', epsilon=0.001, n=n)